# Inference Notebook for Saudi Dialect Text Classifier
This notebook loads the fine-tuned SA-BERT model and runs classification on test samples.
- Model: `AI-Diploma/Najd_Saudi_Classifier_v2`


In [ ]:
!pip install transformers gradio --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load fine-tuned model from Hugging Face Hub
model_name = "AI-Diploma/Najd_Saudi_Classifier_v2"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

## Prediction Function

In [ ]:
def predict_category(text):
    ARABIC_CATEGORIES = {
    "Arts and Media": "الفنون والإعلام",
    "Business and Money": "الأعمال والمال",
    "Cars and Driving": "السيارات والقيادة",
    "Culture and Traditions": "الثقافة والتقاليد",
    "Daily Life": "الحياة اليومية",
    "Days and Dates": "الأيام والتواريخ",
    "Descriptions": "الوصف",
    "Directions": "الاتجاهات",
    "Economy and Finance": "الاقتصاد والمالية",
    "Education and Training": "التعليم والتدريب",
    "Emotions": "المشاعر",
    "Entertainment": "الترفيه",
    "Events and Celebrations": "الأحداث والاحتفالات",
    "Family and Relationships": "الأسرة والعلاقات",
    "Fitness and Exercise": "اللياقة والرياضة",
    "Food and Dining": "الطعام والمأكولات",
    "Greetings": "التحيات",
    "Health": "الصحة",
    "Hobbies and Interests": "الهوايات والاهتمامات",
    "Home and Living": "المنزل والمعيشة",
    "Instructions and Guidelines": "التعليمات والإرشادات",
    "Law and Justice": "القانون والعدالة",
    "Mental Health": "الصحة النفسية",
    "Opinions": "الآراء",
    "Planning and Decisions": "التخطيط والقرارات",
    "Questions": "الأسئلة",
    "Real Estate and Housing": "العقارات والإسكان",
    "Religion and Spirituality": "الدين والروحانيات",
    "Responses": "الردود",
    "Saudi Cities and Regions": "المدن والمناطق السعودية",
    "Shopping": "التسوق",
    "Social Media": "وسائل التواصل الاجتماعي",
    "Sports": "الرياضة",
    "Study and Education": "الدراسة والتعليم",
    "Technology": "التكنولوجيا",
    "Time": "الوقت",
    "Tourism": "السياحة",
    "Transport": "النقل",
    "Travel": "السفر",
    "Weather": "الطقس",
    "Wishes and Dreams": "الأمنيات والأحلام",
    "Work": "العمل"
}

    output = clf(text)[0]
    label = output["label"]
    label = ARABIC_CATEGORIES[label]
    return f"{label}\n"

## Gradio App

In [ ]:
import gradio as gr
import time
import random

custom_css = """
/* Import Arabic-friendly fonts */
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&family=Amiri:wght@400;700&display=swap');

/* Global RTL Styles */
* {
    font-family: 'Amiri', 'Inter', 'Calibri', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
    box-sizing: border-box;
    direction: rtl;
}

/* Professional Dark Blue Background */
.gradio-container {
    background:
        radial-gradient(circle at 20% 80%, rgba(30, 58, 138, 0.08) 0%, transparent 50%),
        radial-gradient(circle at 80% 20%, rgba(37, 99, 235, 0.06) 0%, transparent 50%),
        radial-gradient(circle at 40% 40%, rgba(29, 78, 216, 0.05) 0%, transparent 50%),
        linear-gradient(135deg, #0f172a 0%, #1e293b 25%, #0f172a 50%, #1e293b 75%, #0f172a 100%);
    min-height: 100vh;
    padding: 0;
    font-size: 14px;
    color: #e2e8f0;
    direction: rtl;
}

/* Professional Enhanced Header */
.main-header {
    background:
        linear-gradient(135deg,
            rgba(15, 23, 42, 0.98) 0%,
            rgba(30, 41, 59, 0.96) 25%,
            rgba(15, 23, 42, 0.98) 50%,
            rgba(30, 41, 59, 0.96) 75%,
            rgba(15, 23, 42, 0.98) 100%);
    backdrop-filter: blur(40px);
    border-bottom: 1px solid rgba(59, 130, 246, 0.1);
    color: white;
    padding: 60px 0;
    margin-bottom: 50px;
    position: relative;
    overflow: hidden;
    text-align: center;
}

/* Subtle Professional Background Animation */
.main-header::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background:
        radial-gradient(circle at 25% 25%, rgba(37, 99, 235, 0.08) 0%, transparent 50%),
        radial-gradient(circle at 75% 75%, rgba(29, 78, 216, 0.06) 0%, transparent 50%);
    animation: professionalGlow 8s ease-in-out infinite alternate;
    pointer-events: none;
}

.main-header::after {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background:
        repeating-linear-gradient(
            90deg,
            transparent,
            transparent 98px,
            rgba(59, 130, 246, 0.02) 100px
        );
    pointer-events: none;
}

/* Professional Title Styling */
.title-text {
    font-family: 'Amiri', serif;
    font-size: 48px;
    font-weight: 700;
    text-align: center;
    margin-bottom: 20px;
    line-height: 1.2;
    color: #ffffff;
    position: relative;
    z-index: 3;
    text-shadow:
        0 0 30px rgba(37, 99, 235, 0.3),
        0 2px 15px rgba(0, 0, 0, 0.4);
    filter: drop-shadow(0 4px 20px rgba(29, 78, 216, 0.2));
}

.title-text::before {
    position: absolute;
    top: -20px;
    right: -30px;
    font-size: 24px;
    animation: subtleFloat 4s ease-in-out infinite;
    z-index: 4;
    opacity: 0.8;
}

.title-text::after {
    position: absolute;
    bottom: -20px;
    left: -30px;
    font-size: 20px;
    animation: subtleFloat 4s ease-in-out infinite reverse;
    z-index: 4;
    opacity: 0.8;
}

/* Professional Subtitle */
.subtitle-text {
    font-family: 'Inter', sans-serif;
    font-size: 18px;
    text-align: center;
    font-weight: 400;
    line-height: 1.6;
    position: relative;
    z-index: 3;
    margin-bottom: 30px;
    color: rgba(226, 232, 240, 0.9);
    text-shadow: 0 2px 10px rgba(0, 0, 0, 0.3);
}

/* Subtle Decorative Elements */
.header-decoration {
    position: absolute;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    width: 250px;
    height: 250px;
    border: 1px solid rgba(37, 99, 235, 0.08);
    border-radius: 50%;
    animation: slowRotate 30s linear infinite;
    z-index: 1;
}

.header-decoration::before {
    content: '';
    position: absolute;
    top: 20px;
    left: 20px;
    right: 20px;
    bottom: 20px;
    border: 1px solid rgba(29, 78, 216, 0.06);
    border-radius: 50%;
    animation: slowRotate 25s linear infinite reverse;
}

/* Professional Badge */
.version-badge {
    display: inline-block;
    background: rgba(37, 99, 235, 0.15);
    backdrop-filter: blur(15px);
    border: 1px solid rgba(37, 99, 235, 0.2);
    border-radius: 20px;
    padding: 8px 16px;
    font-size: 12px;
    font-weight: 500;
    color: #93c5fd;
    margin-top: 15px;
    text-shadow: none;
    animation: subtlePulse 3s ease-in-out infinite;
}

/* RTL Content Alignment */
.main-content {
    max-width: 1200px;
    margin: 0 auto;
    padding: 0 32px;
    direction: rtl;
}

/* Professional Input Section */
.input-section {
    background: rgba(30, 41, 59, 0.6);
    backdrop-filter: blur(25px);
    border: 1px solid rgba(59, 130, 246, 0.08);
    border-radius: 16px;
    padding: 32px;
    margin-bottom: 32px;
    box-shadow:
        0 8px 32px rgba(0, 0, 0, 0.4),
        inset 0 1px 0 rgba(255, 255, 255, 0.05);
    position: relative;
    overflow: hidden;
    transition: all 0.3s ease;
    direction: rtl;
}

.input-section::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(135deg, rgba(37, 99, 235, 0.03) 0%, transparent 50%);
    pointer-events: none;
}

.input-section:hover {
    transform: translateY(-2px);
    box-shadow:
        0 12px 40px rgba(0, 0, 0, 0.5),
        inset 0 1px 0 rgba(255, 255, 255, 0.08);
    border-color: rgba(59, 130, 246, 0.12);
}

.section-title {
    font-size: 18px;
    font-weight: 500;
    color: #e2e8f0;
    margin-bottom: 20px;
    display: flex;
    align-items: center;
    gap: 8px;
    text-align: right;
    direction: rtl;
}

.input-container textarea {
    background: rgba(51, 65, 85, 0.4) !important;
    backdrop-filter: blur(15px) !important;
    border: 1px solid rgba(59, 130, 246, 0.08) !important;
    border-radius: 12px !important;
    color: #e2e8f0 !important;
    font-size: 16px !important;
    padding: 16px 20px !important;
    transition: all 0.3s ease !important;
    min-height: 120px !important;
    resize: vertical !important;
    box-shadow: inset 0 2px 10px rgba(0, 0, 0, 0.3) !important;
    direction: rtl !important;
    text-align: right !important;
}

.input-container textarea:focus {
    border-color: rgba(37, 99, 235, 0.3) !important;
    box-shadow:
        0 0 0 1px rgba(37, 99, 235, 0.2) !important,
        inset 0 2px 10px rgba(0, 0, 0, 0.3) !important;
    outline: none !important;
    background: rgba(51, 65, 85, 0.5) !important;
}

.input-container textarea::placeholder {
    color: rgba(226, 232, 240, 0.4) !important;
    text-align: right !important;
}

/* Professional Button Styling */
.button-row {
    display: flex;
    gap: 16px;
    margin-top: 24px;
    direction: rtl;
}

.predict-button {
    background: rgba(71, 85, 105, 0.5) !important;
    backdrop-filter: blur(20px) !important;
    color: #e2e8f0 !important;
    border: 1px solid rgba(59, 130, 246, 0.15) !important;
    padding: 14px 28px !important;
    font-size: 16px !important;
    font-weight: 500 !important;
    border-radius: 10px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
    flex: 2 !important;
    box-shadow:
        0 4px 20px rgba(0, 0, 0, 0.3),
        inset 0 1px 0 rgba(255, 255, 255, 0.05) !important;
    position: relative !important;
    overflow: hidden !important;
    text-align: center !important;
}

.predict-button::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(37, 99, 235, 0.1), transparent);
    transition: left 0.6s ease;
}

.predict-button:hover::before {
    left: 100%;
}

.predict-button:hover {
    transform: translateY(-2px) !important;
    border-color: rgba(37, 99, 235, 0.25) !important;
    background: rgba(71, 85, 105, 0.6) !important;
    box-shadow:
        0 8px 30px rgba(0, 0, 0, 0.4),
        0 0 20px rgba(37, 99, 235, 0.15),
        inset 0 1px 0 rgba(255, 255, 255, 0.08) !important;
}

.clear-button {
    background: rgba(71, 85, 105, 0.3) !important;
    backdrop-filter: blur(20px) !important;
    color: rgba(226, 232, 240, 0.7) !important;
    border: 1px solid rgba(59, 130, 246, 0.08) !important;
    padding: 14px 24px !important;
    font-size: 16px !important;
    font-weight: 500 !important;
    border-radius: 10px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
    flex: 1 !important;
    box-shadow:
        0 4px 20px rgba(0, 0, 0, 0.3),
        inset 0 1px 0 rgba(255, 255, 255, 0.03) !important;
    text-align: center !important;
}

.clear-button:hover {
    transform: translateY(-2px) !important;
    background: rgba(71, 85, 105, 0.4) !important;
    border-color: rgba(59, 130, 246, 0.12) !important;
    color: rgba(226, 232, 240, 0.9) !important;
}

/* Professional Result Section */
.result-section {
    background: rgba(30, 41, 59, 0.6);
    backdrop-filter: blur(25px);
    border: 1px solid rgba(59, 130, 246, 0.08);
    border-radius: 16px;
    padding: 32px;
    margin-bottom: 32px;
    box-shadow:
        0 8px 32px rgba(0, 0, 0, 0.4),
        inset 0 1px 0 rgba(255, 255, 255, 0.05);
    position: relative;
    overflow: hidden;
    transition: all 0.3s ease;
    direction: rtl;
}

.result-section::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(135deg, rgba(37, 99, 235, 0.02) 0%, transparent 50%);
    pointer-events: none;
}

.result-section:hover {
    transform: translateY(-2px);
    box-shadow:
        0 12px 40px rgba(0, 0, 0, 0.5),
        inset 0 1px 0 rgba(255, 255, 255, 0.08);
    border-color: rgba(59, 130, 246, 0.12);
}

.result-title {
    font-size: 18px;
    font-weight: 500;
    color: #e2e8f0;
    margin-bottom: 20px;
    display: flex;
    align-items: center;
    gap: 8px;
    text-align: right;
    direction: rtl;
}

.output-container textarea {
    background: rgba(37, 99, 235, 0.08) !important;
    backdrop-filter: blur(15px) !important;
    border: 1px solid rgba(37, 99, 235, 0.15) !important;
    border-radius: 12px !important;
    color: #e2e8f0 !important;
    font-size: 16px !important;
    padding: 20px !important;
    font-weight: 500 !important;
    min-height: 100px !important;
    resize: none !important;
    box-shadow: inset 0 2px 10px rgba(0, 0, 0, 0.3) !important;
    direction: rtl !important;
    text-align: right !important;
}

/* Professional Examples Section */
.examples-section {
    background: rgba(30, 41, 59, 0.6);
    backdrop-filter: blur(25px);
    border: 1px solid rgba(59, 130, 246, 0.08);
    border-radius: 16px;
    padding: 32px;
    box-shadow:
        0 8px 32px rgba(0, 0, 0, 0.4),
        inset 0 1px 0 rgba(255, 255, 255, 0.05);
    position: relative;
    overflow: hidden;
    transition: all 0.3s ease;
    direction: rtl;
}

.examples-section::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(135deg, rgba(29, 78, 216, 0.02) 0%, transparent 50%);
    pointer-events: none;
}

.examples-section:hover {
    transform: translateY(-2px);
    box-shadow:
        0 12px 40px rgba(0, 0, 0, 0.5),
        inset 0 1px 0 rgba(255, 255, 255, 0.08);
    border-color: rgba(59, 130, 246, 0.12);
}

.examples-title {
    font-size: 18px;
    font-weight: 500;
    color: #e2e8f0;
    margin-bottom: 20px;
    display: flex;
    align-items: center;
    gap: 8px;
    text-align: right;
    direction: rtl;
}

.examples-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
    gap: 16px;
    margin-top: 20px;
    direction: rtl;
}

.example-button {
    background: rgba(71, 85, 105, 0.4) !important;
    backdrop-filter: blur(15px) !important;
    color: rgba(226, 232, 240, 0.8) !important;
    border: 1px solid rgba(59, 130, 246, 0.08) !important;
    padding: 14px 20px !important;
    border-radius: 10px !important;
    font-size: 15px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
    text-align: right !important;
    font-weight: 400 !important;
    line-height: 1.4 !important;
    position: relative !important;
    overflow: hidden !important;
    box-shadow:
        0 2px 10px rgba(0, 0, 0, 0.3),
        inset 0 1px 0 rgba(255, 255, 255, 0.03) !important;
    direction: rtl !important;
}

.example-button::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(37, 99, 235, 0.08), transparent);
    transition: left 0.5s ease;
}

.example-button:hover::before {
    left: 100%;
}

.example-button:hover {
    transform: translateY(-2px) scale(1.02) !important;
    background: rgba(71, 85, 105, 0.5) !important;
    border-color: rgba(59, 130, 246, 0.15) !important;
    color: rgba(226, 232, 240, 0.95) !important;
    box-shadow:
        0 6px 20px rgba(0, 0, 0, 0.4),
        inset 0 1px 0 rgba(255, 255, 255, 0.05) !important;
}

/* RTL Layout */
.main-row {
    display: flex;
    gap: 32px;
    align-items: flex-start;
    direction: rtl;
}

.left-column {
    flex: 2;
}

.right-column {
    flex: 1;
}

/* Professional Animations */
@keyframes professionalGlow {
    0% { opacity: 0.6; }
    100% { opacity: 0.8; }
}

@keyframes subtleFloat {
    0%, 100% { transform: translateY(0px); }
    50% { transform: translateY(-6px); }
}

@keyframes slowRotate {
    from { transform: translate(-50%, -50%) rotate(0deg); }
    to { transform: translate(-50%, -50%) rotate(360deg); }
}

@keyframes subtlePulse {
    0%, 100% { opacity: 0.8; }
    50% { opacity: 0.6; }
}

@keyframes professionalFadeIn {
    from {
        opacity: 0;
        transform: translateY(15px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.fade-in {
    animation: professionalFadeIn 0.5s ease-out;
}

/* Responsive Design */
@media (max-width: 768px) {
    .main-row {
        flex-direction: column;
        gap: 24px;
    }

    .main-content {
        padding: 0 20px;
    }

    .input-section, .result-section, .examples-section {
        padding: 24px;
    }

    .title-text {
        font-size: 36px;
    }

    .subtitle-text {
        font-size: 16px;
    }

    .examples-grid {
        grid-template-columns: 1fr;
    }

    .header-decoration {
        width: 180px;
        height: 180px;
    }
}

/* Professional Focus States */
button:focus,
textarea:focus {
    outline: 2px solid rgba(37, 99, 235, 0.3);
    outline-offset: 2px;
}

/* Professional Scrollbar */
::-webkit-scrollbar {
    width: 8px;
    height: 8px;
}

::-webkit-scrollbar-track {
    background: rgba(30, 41, 59, 0.4);
    border-radius: 4px;
}

::-webkit-scrollbar-thumb {
    background: rgba(71, 85, 105, 0.6);
    border-radius: 4px;
    transition: background 0.3s ease;
}

::-webkit-scrollbar-thumb:hover {
    background: rgba(71, 85, 105, 0.8);
}

/* Enhanced Layer Depth */
.gradio-container > * {
    position: relative;
    z-index: 1;
}

.main-header {
    z-index: 10;
}

.main-content {
    z-index: 5;
}
"""

# Create the enhanced Arabic interface
with gr.Blocks(
    css=custom_css,
    title="Saudi Dialect Category Classifier Using SA-Bert",
    theme=gr.themes.Soft(
        primary_hue="slate",
        neutral_hue="slate",
        font=[gr.themes.GoogleFont("Inter"), "Amiri", "Calibri", "sans-serif"]
    )
) as demo:

    # Enhanced fancy header with RTL support
    with gr.Row():
        with gr.Column():
            gr.HTML("""
                <div class="main-header">
                    <div class="header-decoration"></div>
                    <div class="main-content">
                        <div class="title-text">🇸🇦 Saudi Dialect Category Classifier Using SA-Bert </div>
                        <div class="subtitle-text">نموذج ذكي متطور لتصنيف النصوص السعودية إلى فئات مختلفة باستخدام تقنيات الذكاء الاصطناعي المتقدمة</div>
                    </div>
                </div>
            """)

    # RTL Main content
    with gr.Row():
        with gr.Column():
            gr.HTML('<div class="main-content">')

            # RTL Input and Result sections
            with gr.Row(elem_classes="main-row"):
                with gr.Column(scale=1, elem_classes="right-column"):
                    # RTL Input Section
                    with gr.Group(elem_classes="input-section"):
                        gr.HTML('<div class="section-title">📝 إدخال النص</div>')
                        input_text = gr.Textbox(
                            lines=5,
                            placeholder="اكتب النص المراد تصنيفه هنا...",
                            label="",
                            elem_classes="rtl-text input-container",
                            show_label=False,
                            rtl=True
                        )

                        with gr.Row(elem_classes="button-row"):
                            predict_btn = gr.Button(
                                "تصنيف النص",
                                variant="primary",
                                elem_classes="predict-button"
                            )
                            clear_btn = gr.Button(
                                "مسح",
                                variant="secondary",
                                elem_classes="clear-button"
                            )

                with gr.Column(scale=1, elem_classes="left-column"):
                    # RTL Result Section
                    with gr.Group(elem_classes="result-section"):
                        gr.HTML('<div class="result-title">📊 النتيجة</div>')
                        output_text = gr.Textbox(
                            label="",
                            lines=4,
                            elem_classes="rtl-text output-container fade-in",
                            show_label=False,
                            interactive=False,
                            rtl=True
                        )

            # RTL Examples Section
            with gr.Group(elem_classes="examples-section"):
                gr.HTML('<div class="examples-title">💡 أمثلة للتجربة</div>')

                examples = [
                    "جربت جوال سامسونج الجديد، التصوير فيه خرافي بس البطارية ما أعجبتني",
                    "أخذت مخالفة على سرعة وأنا ما كنت منتبه أصلاً، بس أعترف كنت أسرع شوي",
                    "الأسعار ارتفعت هالفترة، حتى السلع الأساسية مثل الرز والزيت صارت أغلى بكثير من قبل",

                    "صار لي فترة ما أروح النادي، بس بديت أحس بآلام في ظهري، فقلت لازم أرجع أتمرن ولو مشي بسيط يومياً",
                    "كنت أتابع مسلسل تركي على نتفليكس، الأحداث مشوقة جدًا، بس النهاية شوي أحبطتني ما توقعتها كذا"
                ]

                with gr.Row(elem_classes="examples-grid"):
                    example_buttons = []
                    for example in examples:
                        btn = gr.Button(
                            example,
                            elem_classes="example-button",
                            size="sm"
                        )
                        example_buttons.append(btn)

            gr.HTML('</div>')

    # Event handlers
    def predict_with_feedback(text):
        if not text.strip():
            return "يرجى إدخال نص للتصنيف"
        return predict_category(text)

    def clear_all():
        return "", ""

    def set_example(example_text):
        return example_text

    # Event bindings
    predict_btn.click(
        predict_with_feedback,
        inputs=input_text,
        outputs=output_text
    )

    clear_btn.click(
        clear_all,
        outputs=[input_text, output_text]
    )

    # Example button events
    for i, btn in enumerate(example_buttons):
        btn.click(
            set_example,
            inputs=gr.State(examples[i]),
            outputs=input_text
        )

    # Enter key support
    input_text.submit(
        predict_with_feedback,
        inputs=input_text,
        outputs=output_text
    )

# Launch with optimized settings
if __name__ == "__main__":
    demo.launch(
        server_name="0.0.0.0",
        share=True,
        show_error=True,
        inbrowser=True,
        favicon_path=None
    )